In [21]:
!pip install pyspark
!pip install requests

In [22]:
import requests

url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz'
r = requests.get(url, allow_redirects=True)

open('fhv_tripdata_2019-10.csv.gz', 'wb').write(r.content)

19375751

In [23]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('fhv_tripdata') \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv.gz')

df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [14]:
spark.version

'3.5.0'

In [25]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [28]:
import pandas as pd

df_pd = pd.read_csv('fhv_tripdata_2019-10.csv.gz', nrows=1000)
df_schema = spark.createDataFrame(df_pd).schema

df_schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [ ]:
from pyspark.sql import types

schema = types.StructType([
    types.StructField ('dispatching_base_num', types.StringType(), True), 
    types.StructField ('pickup_datetime', types.TimestampType(), True), 
    types.StructField ('dropOff_datetime', types.TimestampType(), True), 
    types.StructField ('PUlocationID', types.IntegerType(), True), 
    types.StructField ('DOlocationID', types.IntegerType(), True), 
    types.StructField ('SR_Flag', types.StringType(), True), 
    types.StructField ('Affiliated_base_number', types.StringType(), True)
])


df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

df = df.repartition(6)

df.write.parquet('fhv_tripdata/2019/10/')

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

count_on_15th_oct = df.filter(F.dayofmonth("pickup_datetime") == 15).count()

print("Number of records with start date on 15th of October:", count_on_15th_oct)

Number of records with start date on 15th of October: 62610


In [35]:
df_with_duration = df.withColumn(
    "trip_duration_hours", 
    (F.col("dropOff_datetime").cast("long") - F.col("pickup_datetime").cast("long")) / 3600
)

max_trip_duration_hours = df_with_duration.agg(F.max("trip_duration_hours")).collect()[0][0]

print("Length of the longest trip in hours:", max_trip_duration_hours)

Length of the longest trip in hours: 631152.5


In [37]:
import requests

url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'
r = requests.get(url, allow_redirects=True)

open('taxi_zone_lookup.csv', 'wb').write(r.content)    

12322

In [38]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [40]:
df.createOrReplaceTempView ("trips")
df_zones.createOrReplaceTempView ("tripZones")

In [45]:
least_trips = spark.sql("""
    SELECT Zone, COUNT(*) as count
    FROM trips
    JOIN tripZones ON trips.PUlocationID = tripZones.LocationID
    GROUP BY Zone
    ORDER BY count ASC
    LIMIT 10
""")
                        
least_trips.show()                        

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|         Jamaica Bay|    1|
|Governor's Island...|    2|
| Green-Wood Cemetery|    5|
|       Broad Channel|    8|
|     Highbridge Park|   14|
|        Battery Park|   15|
|Saint Michaels Ce...|   23|
|Breezy Point/Fort...|   25|
|Marine Park/Floyd...|   26|
|        Astoria Park|   29|
+--------------------+-----+

